# Ejemplo de uso para morfología de cuencas

Coloque los archivos `dem.tif` y `dir.tif` dentro de `examples/data` antes de ejecutar este cuaderno.

In [ ]:
import sys
import os

# Agregar la carpeta raíz del proyecto al PYTHONPATH
sys.path.append(os.path.abspath(".."))  # o la ruta donde está wmf_py

In [ ]:
from pathlib import Path
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from wmf_py.cu_py import basics, metrics

In [ ]:


base = Path("examples/data")
if not base.exists():
    base = Path("data")

dem_path = base / "Dem_2m_palmitas_4326.tif"
dir_path = base / "DIR_2m_palmitas_4326.tif"

with rasterio.open(dem_path) as dem_src:
    dem = dem_src.read(1)
    transform = dem_src.transform  # (a,b,c,d,e,f) = (dx, 0, x0, 0, dy_neg, y0_top)
    nodata_dem = dem_src.nodata

with rasterio.open(dir_path) as dir_src:
    flowdir = dir_src.read(1)
    nodata_dir = dir_src.nodata

# --- 1) Máscara robusta (DEM y flowdir) ---
# Maneja correctamente None y NaN
if nodata_dem is None or (isinstance(nodata_dem, float) and np.isnan(nodata_dem)):
    mask_dem = np.isfinite(dem)
else:
    mask_dem = dem != nodata_dem

if nodata_dir is None or (isinstance(nodata_dir, float) and np.isnan(nodata_dir)):
    mask_dir = np.isfinite(flowdir)
else:
    mask_dir = flowdir != nodata_dir

# En algunos D8 los valores válidos son {1..8}; si aplica, forzamos eso:
mask_dir &= np.isin(flowdir, [1,2,3,4,5,6,7,8])

mask = mask_dem & mask_dir

# --- 2) Parámetros espaciales y helper de coordenadas ---
dx = transform[0]
dy = -transform[4]                    # tamaño de píxel positivo
xll = transform[2]                    # X de la esquina izq
yll = transform[5] + transform[4]*dem.shape[0]  # Y de la esquina inferior (porque e es negativo)

nrows, ncols = dem.shape

def rc_from_xy(x, y):
    # fila/col desde el (xll, yll) y tamaño de celda dx,dy
    c = int((x - xll) // dx)
    r = int((y - yll) // dy)
    return r, c

def xy_center_from_rc(r, c):
    x = xll + (c + 0.5)*dx
    y = yll + (r + 0.5)*dy
    return x, y

def in_bounds(r, c):
    return (0 <= r < nrows) and (0 <= c < ncols)

# --- 3) Outlet inicial (tu idea de 3 celdas) + validación ---
outlet_x = xll + 3*dx + dx/2
outlet_y = yll + 3*dy + dy/2

r0, c0 = rc_from_xy(outlet_x, outlet_y)
if not in_bounds(r0, c0):
    raise ValueError(f"Outlet fuera del raster: r0={r0}, c0={c0}")

# --- 4) Snap al píxel válido más cercano si cae fuera de mask ---
if not mask[r0, c0]:
    # búsqueda incremental en ventanas cuadradas
    max_radius = 20   # ajústalo si tu grid es más grande
    found = False
    best = None
    best_dist2 = None
    x0c, y0c = xy_center_from_rc(r0, c0)
    for rad in range(1, max_radius+1):
        rmin = max(0, r0 - rad); rmax = min(nrows-1, r0 + rad)
        cmin = max(0, c0 - rad); cmax = min(ncols-1, c0 + rad)
        sub = mask[rmin:rmax+1, cmin:cmax+1]
        if sub.any():
            rr, cc = np.where(sub)
            rr += rmin; cc += cmin
            # el más cercano en distancia al centro de celda
            for r, c in zip(rr, cc):
                xC, yC = xy_center_from_rc(r, c)
                d2 = (xC - outlet_x)**2 + (yC - outlet_y)**2
                if (best is None) or (d2 < best_dist2):
                    best = (r, c); best_dist2 = d2
            found = True
            break
    if not found:
        raise ValueError("No encontré un píxel válido cerca del outlet propuesto. Revisa el DEM/flowdir o el punto de salida.")
    r0, c0 = best

# --- 5) Corte de cuenca, acumulación y cauce principal ---
basin = basics.basin_cut(dem, (r0, c0), flowdir, mask)   # ahora sí debería pasar
mask_basin = basin["mask"]
acum = basics.basin_acum(flowdir, mask_basin)

# Umbral para cauces (ajústalo; 1 es muy bajo para 2 m):
thresh = 200  # ~200 celdas contribuyentes; tunea para tu resolución
stream_mask = (acum >= thresh) & mask_basin

# Encuentra y corta el cauce principal
metrics.basin_ppalstream_find(stream_mask, flowdir, dem, dx, dy, (r0, c0))
profile = metrics.basin_ppalstream_cut()

# --- 6) Métricas básicas ---
area_km2 = mask_basin.sum() * dx * dy / 1e6
length_km = profile[1, -1] / 1000.0
slope_pct = np.polyfit(profile[1, ::-1], profile[0], 1)[0] * 100

print(f"Área: {area_km2:.3f} km² | Largo cauce: {length_km:.3f} km | Pendiente media: {slope_pct:.2f} %")


ValueError: operands could not be broadcast together with shapes (1125,769) (1133,769) 

In [10]:
np.shape(flowdir)

(1133, 769)

In [ ]:
print(f"Área de la cuenca: {area_km2:.2f} km²")
print(f"Longitud del cauce principal: {length_km:.2f} km")
print(f"Pendiente media del cauce: {slope_pct:.2f} %")

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(dem, cmap="terrain")
plt.contour(mask_basin, levels=[0.5], colors="blue")
plt.title("Cuenca delineada")
plt.axis("off")

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(mask_basin, cmap="Greys", alpha=0.5)
plt.imshow(np.where(stream_mask, stream_mask, np.nan), cmap="Blues", alpha=0.7)
plt.title("Red de drenaje")
plt.axis("off")

In [ ]:
plt.figure()
plt.plot(profile[1], profile[0])
plt.xlabel("Distancia (m)")
plt.ylabel("Elevación (m)")
plt.title("Perfil longitudinal del cauce principal")
plt.grid(True)